# Nanoparticle Feature Setup <a name="head"></a>

This tutorial details how to generate features from a data set of nanoparticle atoms objects. The data imported here is for 147-atom AuPt alloyed nanoprticles with varying composition and chemical ordering.

## Table of Contents
[(Back to top)](#head)

-   [Requirements](#requirements)
-   [Initialization](#initialization)
-   [Generating Feature Vectors](#generating-feature-vectors)
-   [Storing Data](#storing-data)
-   [Retrieving Date](#retrieving-data)
-   [Conclusions](#conclusions)

## Requirements <a name="requirements"></a>
[(Back to top)](#head)

-   [CatLearn](https://github.com/SUNCAT-Center/CatLearn)
-   [ASE](https://wiki.fysik.dtu.dk/ase/)
-   [numpy](http://www.numpy.org/)

## Initialization <a name="initialization"></a>
[(Back to top)](#head)

To start with we import some functions from ASE and AtoML.

In [ ]:
from IPython.display import Image

import os
import numpy as np
import random

from ase.ga.data import DataConnection
from ase.io import write

from catlearn.api.ase_data_setup import get_unique, get_train
from catlearn.fingerprint.setup import FeatureGenerator
from catlearn.utilities import DescriptorDatabase

## Generating Feature Vectors <a name="generating-feature-vectors"></a>
[(Back to top)](#head)

Some data is imported from an existing `ase.db`.

In [ ]:
# Connect ase atoms database.
gadb = DataConnection('../../data/gadb.db')

# Get all relaxed candidates from the db file.
all_cand = gadb.get_all_relaxed_candidates(use_extinct=False)

The nanoparticles can be displayed with the following.

In [ ]:
write('particle.png', all_cand[-100])
Image('particle.png')

There are a couple of functions that will make some selection of data for testing and training. In reality, it is far more robust to do this with some form of cross-validation, but this will suffice for this tutorial. The `get_unique` function will randomly select a defined data size. The `get_train` can then generate a training dataset with replacement. As the naming suggests, it is ensured none of the data in the unique dataset will make it into the training set.

In [ ]:
testset = get_unique(atoms=all_cand, size=100, key='raw_score')

trainset = get_train(atoms=all_cand, size=500, taken=testset['taken'],
                     key='raw_score')

Once the atoms objects have been prepared, it is possible to generate the feature sets. This can be performed in parallel by setting `nprocs` greater than 1. If `None` is set then all available cores will be utilized. Two functions can be called to generate the feature sets `return_vec` and the feature names `return_names`.

In [ ]:
generator = FeatureGenerator(atom_types=[78, 79], nprocs=1)
data = generator.return_vec(trainset['atoms'], [generator.eigenspectrum_vec])
vec_names = generator.return_names([generator.eigenspectrum_vec])

## Storing Data <a name="storing-data"></a>
[(Back to top)](#head)

Once the features have been generated, it is possible to store them in a database. This initial implementation of the db isn't so optimal but is a simple way of temporarily storing all the feature vectors. The following lines of code will initialize the db.

In [ ]:
# Define variables for database to store system descriptors.
descriptors = vec_names
targets = ['Energy']
column_names = descriptors + targets

# Set up the database to save system descriptors.
dd = DescriptorDatabase(db_name='vec_store.sqlite')
dd.create_db(names=column_names)

The code below will then reformat the data slightly, before storing it in the db. In this case the targets are stored in the `atoms.info['key_value_pairs']['raw_score']`. To keep track of where the feature vectors come from, the atoms UUID is utilized. This is stored in `atoms.info['unique_id']`.

In [ ]:
# Put data in correct format to be inserted into database.
new_data = []
for d, a in zip(data, trainset['atoms']):
    new_data.append([a.info['unique_id']] + list(d) +
    [a.info['key_value_pairs']['raw_score']])

# Fill the database with the data.
dd.fill_db(descriptor_names=column_names, data=new_data)

It is possible to update the db as required. In the following, we just append some random variables from to the db.

In [ ]:
dd.create_column(new_column=['random'])

# Add new name to list of descriptors.
column_names += ['random']

for i in dd.query_db(names=['uuid']):
    dd.update_descriptor(
        descriptor='random', new_data=random.random(), unique_id=i[0])

## Retrieving Date <a name="retrieving-data"></a>
[(Back to top)](#head)

To check what data is now stored in the db, the following can be used.

In [ ]:
stored = dd.get_column_names()
print('\nretrieved column names:\n{}'.format(stored))

To retrieve the features and targets, the following can be used.

In [ ]:
# Test out the database functions.
train_fingerprint = dd.query_db(names=descriptors)
train_target = dd.query_db(names=targets)

print('\nfeature data for candidates: \n{}'.format(train_fingerprint[:10, :]))
print('\ntarget data for candidates: \n{}'.format(train_target[:10, :]))

Finally, the folder can be tidied up.

In [ ]:
os.remove('vec_store.sqlite')
os.remove('particle.png')

## Conclusions <a name="conclusions"></a>
[(Back to top)](#head)

This tutorial has demonstrated how to generate feature vectors for nanoparticle atoms objects and store that data in a temporary db.